# The goal
This is an example on how to initialize the T5 based model that accepts dynamic labels. 
We will load the Atis dataset and then classify using it

## Initialize model

In [1]:
from open_intent_classifier.model import IntentClassifier

model = IntentClassifier(device="cuda", verbose=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## Initialize dataset

In [2]:
import pandas as pd
from datasets import load_dataset

# we don't include any multi-label instances in order to simplify this classification

ATIS_INTENT_MAPPING = {
    'abbreviation': "Abbreviation and Fare Code Meaning Inquiry",
    'aircraft': "Aircraft Type Inquiry",
    'airfare': "Airfare and Fares Questions",
    'airline': "Airline Information Request",
    'airport': "Airport Information and Queries",
    'capacity': "Aircraft Seating Capacity Inquiry",
    'cheapest': "Cheapest Fare Inquiry",
    'city': "Airport Location Inquiry",
    'distance': "Airport Distance Inquiry",
    'flight': "Flight Booking Request",
    'flight_no': "Flight Number Inquiry",
    'flight_time': "Time Inquiry",
    'ground_fare': "Ground Transportation Cost Inquiry",
    'ground_service': "Ground Transportation Inquiry",
    'ground_service+ground_fare': "Airport Ground Transportation and Cost Query",
    'meal': "Inquiry about In-flight Meals",
    'quantity': "Flight Quantity Inquiry",
    'restriction': "Flight Restriction Inquiry"
}

def atis_convert_old_label_to_class_atis(old_label: str):
    if old_label in ATIS_INTENT_MAPPING:
        return ATIS_INTENT_MAPPING[old_label]
    return None


dataset = load_dataset("tuetschek/atis")
dataset.set_format(type="pandas")

df_train: pd.DataFrame = dataset["train"][:]
df_test: pd.DataFrame = dataset["test"][:]

df_train["label"] = df_train["intent"].apply(lambda label: atis_convert_old_label_to_class_atis(label))
df_test["label"] = df_test["intent"].apply(lambda label: atis_convert_old_label_to_class_atis(label))
df_train.dropna(subset=["label"], inplace=True)
df_test.dropna(subset=["label"], inplace=True)

datasets - INFO - PyTorch version 2.4.1 available.


## Sort labels alphabetically - emperically improves performance for Atis dataset

In [3]:
labels = ATIS_INTENT_MAPPING.values()
sorted_labels = sorted(labels)
sorted_labels


['Abbreviation and Fare Code Meaning Inquiry',
 'Aircraft Seating Capacity Inquiry',
 'Aircraft Type Inquiry',
 'Airfare and Fares Questions',
 'Airline Information Request',
 'Airport Distance Inquiry',
 'Airport Ground Transportation and Cost Query',
 'Airport Information and Queries',
 'Airport Location Inquiry',
 'Cheapest Fare Inquiry',
 'Flight Booking Request',
 'Flight Number Inquiry',
 'Flight Quantity Inquiry',
 'Flight Restriction Inquiry',
 'Ground Transportation Cost Inquiry',
 'Ground Transportation Inquiry',
 'Inquiry about In-flight Meals',
 'Time Inquiry']

## Run predictions

In [4]:
from tqdm import tqdm
predictions = []

for index, row in tqdm(df_test.iterrows()):
    text = row["text"]
    prediction = model.predict(text, sorted_labels)
    predictions.append(prediction)

0it [00:00, ?it/s]/home/serj/dev/open-intent-classifier/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
open_intent_classifier.model - DEBUG - Full prompt: Topic %% Customer: i would like to find a flight from charlotte to las vegas that makes a stop in st. louis.
END MESSAGE
Choose one topic that matches customer's issue.
 Options:
# Abbreviation and Fare Code Meaning Inquiry 
# Aircraft Seating Capacity Inquiry 
# Aircraft Type Inquiry 
# Airfare and Fares Questions 
# Airline Information Request 
# Airport Distance Inquiry 
# Airport Ground Transportation and Cost Query 
# Airport Information and Queries 
# Airport Location Inquiry 
# Cheapest Fare Inquiry 
# Flight Booking Request 
# Flight Number Inquiry 
# Flight Quantity Inquiry 
# Flight Restriction Inqu

In [10]:
from sklearn.metrics import classification_report
predictions = [pred.lower() for pred in predictions]
y = [label.lower() for label in df_test["label"].to_list()]

df_classification_report = pd.DataFrame(classification_report(y, predictions, output_dict=True)).T
df_classification_report

/home/serj/dev/open-intent-classifier/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/serj/dev/open-intent-classifier/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/serj/dev/open-intent-classifier/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

,precision,recall,f1-score,support
abbreviation and fare code meaning inquiry,1.000000,1.000000,1.000000,33.000000
aircraft seating capacity inquiry,1.000000,0.952381,0.975610,21.000000
aircraft type inquiry,0.562500,1.000000,0.720000,9.000000
airfare and fares questions,0.500000,0.208333,0.294118,48.000000
airline information request,0.808511,1.000000,0.894118,38.000000
airport distance inquiry,0.833333,1.000000,0.909091,10.000000
airport ground transportation and cost query,0.000000,0.000000,0.000000,0.000000
airport information and queries,0.640000,0.888889,0.744186,18.000000
airport location inquiry,0.307692,0.666667,0.421053,6.000000
airport transportation cost inquiry,0.000000,0.000000,0.000000,0.000000
